In [1]:

import sys
sys.path.append("../")

import os


from desci_sense.twitter import scrape_tweet

from desci_sense.postprocessing.output_parsers import TagTypeParser

In [17]:
TEST_TWEET_W_REF = "https://twitter.com/victorveitch/status/1722300572554969090"
TEST_TWEET_W_REF_2 = "https://twitter.com/maksym_andr/status/1722235666724192688"
TEST_TWEET_WO_REF_1 = "https://twitter.com/mpshanahan/status/1722283975450722407"

In [28]:
import re

# based on ChatGPT and https://stackoverflow.com/a/6041965
def extract_urls(text):
    url_regex = r'((http|ftp|https):\/\/([\w_-]+(?:(?:\.[\w_-]+)+))([\w.,@?^=%&:\/~+#-]*[\w@?^=%&\/~+#-]))'
    res = re.findall(url_regex, text)
    final_res = [r[0] for r in res]
    return final_res


In [22]:
TEST_CASES = [

    ("https://twitter.com/victorveitch/status/1722300572554969090", True),
    ("https://twitter.com/maksym_andr/status/1722235666724192688", True),
    ("https://twitter.com/mpshanahan/status/1722283975450722407", False),
    ("https://twitter.com/victorveitch/status/1722303746397409698", False)
]

In [18]:
tweet = scrape_tweet(TEST_TWEET_WO_REF_1)
tweet

{'conversationID': '1722283973433291133',
 'date': 'Wed Nov 08 16:04:32 +0000 2023',
 'date_epoch': 1699459472,
 'hashtags': [],
 'likes': 3,
 'mediaURLs': [],
 'media_extended': [],
 'possibly_sensitive': False,
 'qrtURL': None,
 'replies': 2,
 'retweets': 0,
 'text': "It's about how we can use the concept of role play to better understand large language models and dialogue agents, such as #ChatGPT and @Google's Bard. #AI\n2/",
 'tweetID': '1722283975450722407',
 'tweetURL': 'https://twitter.com/mpshanahan/status/1722283975450722407',
 'user_name': 'Murray Shanahan',
 'user_screen_name': 'mpshanahan'}

In [19]:
print(tweet["text"])

It's about how we can use the concept of role play to better understand large language models and dialogue agents, such as #ChatGPT and @Google's Bard. #AI
2/


In [29]:
for url, has_url in TEST_CASES:
    tweet = scrape_tweet(url)
    print(extract_urls(tweet["text"]))

['https://t.co/8WXQVI4WBb']
['https://arxiv.org/abs/2311.01906', 'https://t.co/PlPRQVL9yc']
[]


In [6]:
import requests

# Short URL for a Python Requests Tutorial
short_url = 'https://t.co/8WXQVI4WBb'

res = requests.head(short_url)
if res.status_code == 303: # "See Other"
    full_url = res.headers['location']
elif res.status_code == 200: # "OK"
    # let's conclude that short_url is already what we are looking for
    full_url = short_url
else:
    # replace by your error handling:
    assert(False)

AssertionError: 

In [37]:

long_url = 'https://stackoverflow.com/questions/65097231/resolve-masked-shortened-url-twint-is-scraping-from-twitter'
short_url = 'https://t.co/8WXQVI4WBb'

resp = requests.head(short_url)
resp.status_code
true_url = resp.headers["Location"]
true_url

'https://twitter.com/victorveitch/status/1722300572554969090/photo/1'

In [38]:
resp.status_code

301

In [25]:
import re

def extract_urls(text):
    url_regex = r'https?://(?:[-\w.]|(?:%[\da-fA-F]{2}))+'
    return re.findall(url_regex, text)

# Example usage:
text_with_urls = "Here are some URLs: https://www.example.com, http://example.org, and https://www.example.net/path/to/page.html"
urls = extract_urls(text_with_urls)
print(urls)

['https://www.example.com', 'http://example.org', 'https://www.example.net']


In [41]:
import requests

def unshorten_url(url):
    try:
        response = requests.head(url, allow_redirects=True)
        return response.url
    except requests.RequestException as e:
        print(f"Request failed: {e}")
        return url

# Example usage:
short_url = "https://t.co/8WXQVI4WBb"

long__test_url = 'https://twitter.com/victorveitch/status/1722300572554969090'
long_url = unshorten_url(long__test_url)
print(long_url)

https://twitter.com/victorveitch/status/1722300572554969090


In [42]:
def extract_urls(text):
    """ takes a string text as input and uses the regular expression pattern to find all 
    occurrences of URLs in the text. returns a list of all non-overlapping matches of the regular expression pattern in the string.
    """
    url_regex = r'((http|ftp|https):\/\/([\w_-]+(?:(?:\.[\w_-]+)+))([\w.,@?^=%&:\/~+#-]*[\w@?^=%&\/~+#-]))'
    res = re.findall(url_regex, text)
    final_res = [r[0] for r in res]
    return final_res


def unshorten_url(url):
    try:
        response = requests.head(url, allow_redirects=True)
        return response.url
    except requests.RequestException as e:
        # return original url in case of errors
        return url
    

def extract_and_expand_urls(text):
    """_summary_

    Args:
        text (_type_): _description_
    """

    expanded_urls = [unshorten_url(url) for url in extract_urls(text)]
    return expanded_urls

In [43]:
for url, has_url in TEST_CASES:
    tweet = scrape_tweet(url)
    print(extract_and_expand_urls(tweet["text"]))

['https://twitter.com/victorveitch/status/1722300572554969090/photo/1']
['https://arxiv.org/abs/2311.01906', 'https://twitter.com/maksym_andr/status/1722235666724192688/photo/1']
[]
